In [1]:
import google.generativeai as genai
import os
import traceback
import ast


import subprocess
import json
# Configura la API con tu clave
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Definir la ruta del archivo de video
video_path = "IA.mp4"

# Verificar si el archivo existe
if not os.path.exists(video_path):
    print(f"Error: El archivo '{video_path}' no existe.")
    exit()

try:
    uploaded_video = genai.upload_file(video_path)
except Exception as e:
    print(f"Error al cargar el video: {str(e)}")
    exit()

# Selecciona el modelo Gemini 1.5 Pro

def extract_python_code(response_text):
    if "```python" in response_text:
        return response_text.split("```python")[1].split("```")[0].strip()
    elif "```" in response_text:
        return response_text.split("```")[1].split("```")[0].strip()
    else:
        return response_text.strip()

def execute_code_safely(code):
    try:
        exec(code, globals())
        return None  # Sin errores
    except Exception as e:
        error_traceback = traceback.format_exc()
        return error_traceback

def is_code_valid(code):
    try:
        ast.parse(code)
        return True
    except SyntaxError as e:
        return False

# Obtener metadatos del video
def get_video_metadata(video_path):
    try:
        result = subprocess.run(
            ["ffprobe", "-v", "quiet", "-print_format", "json", "-show_format", "-show_streams", video_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT
        )
        return json.loads(result.stdout)
    except Exception as e:
        print(f"Error al obtener metadatos del video: {str(e)}")
        return None

metadata = get_video_metadata(video_path)

if metadata:
    video_duration = float(metadata['format']['duration'])
    video_info = f"El video tiene una duración de {video_duration} segundos."
else:
    video_info = "No se pudo obtener información detallada del video."
    video_duration = None
model = genai.GenerativeModel(model_name="gemini-1.5-pro-exp-0827")

c:\Users\worke\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Error al obtener metadatos del video: [WinError 2] El sistema no puede encontrar el archivo especificado


In [3]:
instrucciones_usuario = input("Describe las ediciones: ")
prompt = f"""
Información del video:
{video_info}

Eres un asistente de edición de video con Python usando MoviePy. Tu tarea es:

1. Analizar el video proporcionado y extraer información relevante que pueda ayudar en las ediciones solicitadas.

2. Basándote en el análisis del video y en las siguientes instrucciones del usuario, genera un script Python que realice las ediciones solicitadas de manera precisa y correcta:

{instrucciones_usuario}

El script debe:
-Configurar explícitamente la ruta de ImageMagick al inicio del script.
     from moviepy.config import change_settings
     # ... código existente ...

# Configuración de la ruta del binario de ImageMagick 


# ... código existente ...
- Utilizar la información obtenida del análisis del video para ajustar las ediciones.
- Verificar que todos los tiempos y posiciones utilizados son válidos y están dentro de los límites del video.
- Inicializar todas las variables antes de usarlas.
- Manejar excepciones con bloques try-except.
- Incluir comentarios explicativos.
- Proporcionar un código limpio y funcional.

Genera únicamente el código Python completo y funcional, sin explicaciones adicionales.
"""
# Primero, generamos el contenido solo con el prompt
response = model.generate_content(prompt)
# Extraer el código Python de la respuesta
python_code = extract_python_code(response.text)

# Bucle de corrección automática
max_attempts = 3
attempt = 0
while attempt < max_attempts:
    # Validar sintaxis del código
    if not is_code_valid(python_code):
        print("El código generado tiene errores de sintaxis.")
        error_message = "Error de sintaxis en el código generado."
    else:
        error_message = execute_code_safely(python_code)

    if not error_message:
        print("El código se ha ejecutado correctamente.")
        break
    else:
        print(f"Error en el intento {attempt + 1}: {error_message}")
        # Enviar el error al modelo para corrección
        correction_prompt = f"""
        El siguiente código ha generado un error durante su ejecución:

        ```python
        {python_code}
        ```

        El mensaje de error es:

        ```
        {error_message}
        ```

        Por favor, corrige el código para solucionar este error, asegurándote de que:

        - Utilizas correctamente la información obtenida del análisis del video.
        - Todas las variables están inicializadas.
        - Los tiempos y posiciones no exceden los límites del video.
        - No hay errores en operaciones matemáticas.

        Proporciona únicamente el código corregido.
        """
        correction_response = model.generate_content([correction_prompt], uploaded_video)
        python_code = extract_python_code(correction_response.text)
        attempt += 1
if attempt == max_attempts:
    print("No se pudo generar un código válido. Por favor, revisa tus instrucciones y vuelve a intentarlo.")
else:
    # Guardar el código corregido
    output_file = "editar_video.py"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(python_code)
    print(f"Se ha creado el archivo '{output_file}' con el código para editar el video según tus instrucciones.")

Error: MoviePy error: the file video.mp4 could not be found!
Please check that you entered the correct path.
El código se ha ejecutado correctamente.
Se ha creado el archivo 'editar_video.py' con el código para editar el video según tus instrucciones.
